In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install -Uq transformers rich[jupyter] sentencepiece

     |████████████████████████████████| 2.6 MB 5.1 MB/s 
     |████████████████████████████████| 209 kB 50.1 MB/s 
     |████████████████████████████████| 1.2 MB 56.3 MB/s 
     |████████████████████████████████| 636 kB 73.9 MB/s 
     |████████████████████████████████| 895 kB 59.0 MB/s 
     |████████████████████████████████| 3.3 MB 25.8 MB/s 
     |████████████████████████████████| 51 kB 7.7 MB/s 


In [126]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

In [127]:
dataset_headline = pd.read_csv('/content/drive/Shareddrives/Всеволод/one_hot_hack/dataset/dataset_public_headline.csv')
dataset_headline.rename({'y': 'title', "X": "headline"}, inplace=True, axis=1)

max_words = 5
for w in range(1, max_words + 1):
    dataset_headline[f"{w}_words"] = dataset_headline['headline'].apply(lambda h: ' '.join(h.split(' ')[:w]))

df = dataset_headline
titles = df['title'].unique()

result = {
    'title': list(),
    'result': list(),
}

c = 1.4
coeffs = [0.1 * c, 0.15 * c, 0.2 * c, 0.15 * c, 0.1 * c]

for title in titles:
    samples = df[df['title'] == title]
    sum_samples = len(samples)
    for w in reversed(range(1, max_words + 1)):
        u = len(samples[f'{w}_words'].unique())
        coeff = u / sum_samples / np.log(sum_samples)
        res = list(samples[f'{w}_words'].value_counts().items())[0][0]
        if coeff <= coeffs[w - 1] and res.split(' ')[-1].lower() not in ['на', 'в']:
            result['title'].append(title)
            if res[-1] == ':':
                res = res[:-1]
            
            result['result'].append(res)
            break


easy_result = pd.DataFrame(result)

In [128]:
easy_result

,title,result
0,Значения фондовых индексов,Значения основных фондовых индексов
1,Рынок акций РФ,Рынок акций РФ утром
2,Кросс-курсы мировых валют,Кросс-курсы основных мировых валют
3,Вакцина от COVID-19,Бахрейн стал первой
4,Курсы валют,"Курс доллара расчетами ""завтра"""
5,Кросс-курсы валют,Кросс-курсы основных мировых валют
6,Итоги торгов акциями на Московской бирже,Результаты торгов акциями
7,Значения индексов Мосбиржи и РТС,Значения индексов Мосбиржи и РТС
8,Торги ADR и GDR российских компаний,Торги ADR и GDR российских
9,Кросс-курсы основных мировых валют,Кросс-курсы основных мировых валют


In [129]:
MAX_SOURCE_TEXT_LENGTH = int(512 * 1)
MAX_TARGET_TEXT_LENGTH = 17

In [130]:


# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console = Console(record=True)

# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""

    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )

    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])

    console.print(table)

# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)

# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [131]:
np.random.seed(42)

train_df_public = pd.read_csv('/content/drive/Shareddrives/Всеволод/one_hot_hack/dataset/dataset_public_joined.csv')
print(len(train_df_public))
train_df_public.drop(train_df_public[train_df_public['y'].isin(easy_result['title'].unique())].index, inplace=True)
print(len(train_df_public))

titles_df = pd.read_csv('/content/drive/Shareddrives/Всеволод/one_hot_hack/dataset/titles.csv')
for _, old_title, new_title in titles_df.itertuples():
  if old_title != new_title:
    train_df_public['y'].replace(old_title, new_title, inplace=True)
    print(old_title)

train_df_our = pd.read_csv('/content/drive/Shareddrives/Всеволод/one_hot_hack/dataset/dataset_our_joined.csv')

train_size = 0.8
train_df_public['is_valid'] = True
unique_y = train_df_public["y"].unique()
train_ys = np.random.choice(unique_y, int(len(unique_y) * train_size), replace=False)
train_df_public.loc[train_df_public[train_df_public['y'].isin(train_ys) == True].index, 'is_valid'] = False

train_df_our['is_valid'] = False
train_df = pd.concat([train_df_public, train_df_our], ignore_index=True, axis=0)

1397
892
Взрыве газа в доме в Зеленодольске
Встрече Блинкена с делегацией КНР
Макроэкономических показатели
Заражений COVID-19
Памятнике на Лубянке
Причастности РФ к кибератакам
Арестовал гендиректора "Архоблэнерго"
Вакцинацию от коронавируса
Инвестиции Лукойла  в upstream


In [132]:
ys = train_df["y"].unique()
amount = 5
for y in ys:
  y_rows = train_df[train_df["y"] == y]
  y_rows = y_rows[amount:]
  #print(y_rows[amount:])
  train_df.drop(y_rows.index, inplace=True)
  if (y[-5:] == " Фото"):
    train_df.replace(y, y[:-5], inplace=True)

In [133]:
sum(train_df['is_valid'])

131

In [134]:
len(train_df)

1022

In [135]:
train_df[train_df['y'] == "Олимпиада-2020"]

,X,y,is_valid
896,FIG признала объективным судейство художествен...,Олимпиада-2020,False


In [136]:
class NewsDataset(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model

    """

    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class

        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]

    def __len__(self):
        """returns the length of dataframe"""

        return len(self.target_text)

    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""

        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])

        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())

        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()

        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }

In [137]:
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]

        if _ % 100 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [138]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=MAX_TARGET_TEXT_LENGTH, 
              num_beams=2,
              repetition_penalty=1.0, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [139]:
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./outputs/"
):

    """
    T5 trainer

    """

    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True

    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)

    # logging
    console.log(f"[Data]: Reading data...\n")

    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text, 'is_valid']]
    display_df(dataframe.head(2))

    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_dataset = dataframe[dataframe["is_valid"] == False].drop(columns="is_valid").reset_index(drop=True)
    val_dataset = dataframe[dataframe["is_valid"] == True].drop(columns="is_valid").reset_index(drop=True)


    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")

    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = NewsDataset(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = NewsDataset(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )

    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }

    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }

    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )

    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")

    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)

    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)

    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions.csv"))

    console.save_text(os.path.join(output_dir, "logs.txt"))

    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [140]:
# let's define model parameters specific to T5
model_params = {
    "MODEL": "cointegrated/rut5-base-multitask",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TRAIN_EPOCHS": 8,  # number of training epochs
    "VAL_EPOCHS": 2,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": MAX_SOURCE_TEXT_LENGTH,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": MAX_TARGET_TEXT_LENGTH,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}

In [141]:
train_df["X"] = "headline | " + train_df["X"]

In [142]:
T5Trainer(
    dataframe=train_df,
    source_text="X",
    target_text="y",
    model_params=model_params,
    output_dir="outputs_headlines",
)

[14:13:20] [Model]: Loading                               <ipython-input-139-6b650c00c30d>:16
           cointegrated/rut5-base-multitask...

[14:13:23] [Data]: Reading data...                        <ipython-input-139-6b650c00c30d>:27

                                         Sample Data                                         
+-------------------------------------------------------------------------------------------+
|                            source_text                              |     target_text     |
|---------------------------------------------------------------------+---------------------|
|headline | Онкология, диабет и инфаркт не являются противопоказанием | Вакцина НИИ "Вектор"|
|    против вакцинации от COVID-19 - глава екатеринбургского НИИ      |                     |
| "Вектор". Москва. 29 марта. ИНТЕРФАКС - Хроническим больным, в том  |                     |
|   числе с онкологией, после инфаркта и с высоким давлением можно    |                     |
|    прививаться от коронавируса COVID-19, в частности, вакциной      |                     |
|  "ЭпиВакКороной", если заболевание находится не в активной форме,   |                     |
|  сообщил вирусолог, иммунолог, руководитель екатеринбургского НИИ   |                     |
|   вирусных инфекций "Вектор" Роспотребнадзора Александр Семенов.    |                     |
|   "Безусловно. Если сейчас процесс у вас не активный, если вы в     |                     |
|   состоянии ремиссии, вне острого процесса, то не просто можно -    |                     |
|нужно, потому что любая инфекция - она только ослабит ваш организм,  |                     |
|   и даст раку шансы на развитие", - сказал Семенов в эфире "Эхо     |                     |
|     Москвы" в понедельник, отвечая на соответствующий вопрос.       |                     |
|Аналогично он ответил на вопрос, можно ли делать прививку в возрасте |                     |
|за 60 лет после двух перенесенных инфарктов. "Если у человека сейчас |                     |
|все в порядке - давление под контролем, общие показатели нормальные, |                     |
|        опять-таки - вне острого процесса, (...) если вы уже         |                     |
|  реабилитировались после инфаркта, то - пожалуйста, прививайтесь,   |                     |
|  ничего страшного с вами не случится, максимум, что у вас будет -   |                     |
|небольшое повышение температуры в течение одного дня, которое легко  |                     |
|                   купируется", - сказал Семенов.                    |                     |
|В целом, по его словам, если хроническое заболевание находится не в  |                     |
|      острой форме, то противопоказаний против вакцинации нет.       |                     |
|"Какое бы заболевание ни было, например, диабет, "если у вас сахар в |                     |
|   целевых показателях, то есть вы принимаете вовремя лекарства,     |                     |
|контролируете - нет противопоказаний. Особенно пептидными вакцинами  |                     |
|  ("ЭпиВакКорона" относится к ним - ИФ), они в этом плане наименее   |                     |
| реактогенные. Если у вас гипертония, и вы вовремя принимаете свое   |                     |
|   гипотензивное средство, контролируете свое давление - идите и     |                     |
|    прививайтесь, ничего страшного не будет", - сказал Семенов.      |                     |
|То же относится, например, к ДЦП, "причем никаких противопоказаний,  |                     |
|прекрасно деточки с ДЦП вырабатывают антитела, потому что опять-таки |                     |
|       это проблема неврологическая, а не иммунологическая".         |                     |
|  По его словам, вакцинация станет сезонной, и со временем вакцину   |                     |
| можно будет более точно подбирать, исходя из особенностей здоровья  |                     |
|   человека, но "сейчас не до выбора". "В условиях эпидемического    |                     |
|   процесса, когда, как сейчас, идет эпидемия, я рекомендую любую    |                     |
|   доступную вакцину, которая есть у вас в вашей поликлинике или     |                     |
|  фельдшерско-акушерском пункте, которая способна о

FULL Dataset: (1022, 3)

TRAIN Dataset: (891, 2)

...

Completed 10

[14:22:47] [Validation Completed.]                       <ipython-input-139-6b650c00c30d>:104

[Model] Model saved @ outputs_headlines/model_files

[Validation] Generation on Validation data saved @ outputs_headlines/predictions.csv

[Logs] Logs saved @ outputs_headlines/logs.txt

In [143]:
predictions = pd.read_csv('outputs_headlines/predictions.csv', index_col=0)
display_df(predictions)


                                         Sample Data                                         
+-------------------------------------------------------------------------------------------+
|                source_text                  |                 target_text                 |
|---------------------------------------------+---------------------------------------------|
|Парламентские консультации досрочные выборы  |        Политический кризис в Молдавии       |
|                   Гросу                     |        Политический кризис в Молдавии       |
|             Парламент Молдавии              |        Политический кризис в Молдавии       |
|             Парламент Молдавии              |        Политический кризис в Молдавии       |
|       Блокировка спутников Starlink         |           Запуск спутников SpaceX           |
|       Блокировка спутников Starlink         |           Запуск спутников SpaceX           |
|       Блокировка спутников Starlink         |           Запуск спутников SpaceX           |
|             Блокировка орбиты               |           Запуск спутников SpaceX           |
|       Блокировка спутников Starlink         |           Запуск спутников SpaceX           |
|  Количество заражений COVID-19 в Молдавии   |             COVID-19 в Молдавии             |
|     В Молдавии новых случаев COVID-19       |             COVID-19 в Молдавии             |
|     В Молдавии новых случаев COVID-19       |             COVID-19 в Молдавии             |
|  Изменения законодательства о экспорте на   |        Экспортные контракты на зерно        |
|                   зерно                     |                                             |
|             "Московская биржа"              |        Экспортные контракты на зерно        |
|              Российская биржа               |        Экспортные контракты на зерно        |
|        Изменения льготы на экспорт          |        Экспортные контракты на зерно        |
|        Протесты на акции 31 января          | Участники несогласованной акции в Петербурге|
|             Акции в Петербурге              | Участники несогласованной акции в Петербурге|
|          Акции на Сенной площади            | Участники несогласованной акции в Петербурге|
|               Авиасообщение                 |  Авиасообщение РФ с Белоруссией и Арменией  |
|      Блокировка российских аэропортов       |  Авиасообщение РФ с Белоруссией и Арменией  |
|      Блокировка российских аэропортов       |  Авиасообщение РФ с Белоруссией и Арменией  |
|           Большой землетрясения             |           Землетрясение в Монголии          |
|            Второе землетрясение             |           Землетрясение в Монголии          |
|           Взрывы на юго-востоке             |           Землетрясение в Монголии          |
|   Большой землетрясения на ГЭС Иркутска     |           Землетрясение в Монголии          |
|           Большой землетрясения             |           Землетрясение в Монголии          |
|         Блокировка аккаунта Трампа          |               Трамп в соцсетях              |
|         Блокировка аккаунта Трампа          |               Трамп в соцсетях              |
|             Блокировка Twitter              |               Трамп в соцсетях              |
|   Внутриполитический кризис в Вашингтоне    |               Трамп в соцсетях              |
|     Блокировка российских газопроводов      |        Аварии на магистралях Газпрома       |
|     Блокировка газопровода в Саранске       |        Аварии на магистралях Газпрома       |
|  Блокировка газопровода в Новосергиевской   |        Аварии на магистралях Газпрома       |
|                  области                    |                                             |
|           Блокировка газопровода            |        Аварии на магистралях Газпрома       |
|    Блокировка газопроводов в Оренбуржье     |        Аварии на магистралях Газпрома       |
|          Козлов из изолятора МВД            |     

In [52]:
!cp -r outputs_headlines/ /content/drive/Shareddrives/Всеволод/one_hot_hack/models/1

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")
# model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-multitask")
PATH = './outputs_headlines/model_files/spiece.model'

model = torch.load(PATH)

def generate(text, **kwargs):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs, num_beams=5, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

UnpicklingError: ignored

In [ ]:
print(generate('headline | Каждый охотник желает знать, где сидит фазан.'))

NameError: ignored

In [ ]:
!rm -r outputs

In [23]:
train_df["y"].unique()

array(['Вакцина НИИ "Вектор"', 'Политический кризис в Молдавии',
       'Запуск спутников SpaceX', 'Значения фондовых индексов',
       'Рынок акций РФ', 'Сделка Яндекс.Такси и Везет',
       'Обстоятельства госпитализации Навального',
       'Кросс-курсы мировых валют', 'Встреча по Афганистану в Москве',
       'Задержания на форуме муниципальных депутатов',
       'Вакцина от COVID-19', 'Курсы валют', 'Кросс-курсы валют',
       'Рынок российских облигаций', 'Юнипро', 'Сессия Совета МАГАТЭ',
       'X5', 'Итоги торгов акциями на Московской бирже',
       'Оборонная выставка IDEX-2021', 'Отключение БелАЭС',
       'Значения индексов Мосбиржи и РТС',
       'Торги ADR и GDR российских компаний', 'Финансовый отчет Роснефти',
       'COVID-19 в Молдавии', 'Снегопад в Москве',
       'Существенные события банков', ' Аварии на ТЭЦ Норникеля',
       'Курс рубля', 'Кросс-курсы основных мировых валют',
       'Заявления Байдена', 'Экспортные контракты на зерно',
       'Участники несогласова

In [24]:
train_df[train_df["y"] == "Заражений COVID-19"]

,X,y
889,headline | Суточный прирост новых случаев COVI...,Заражений COVID-19
890,headline | Число заболевших COVID-19 в Москве ...,Заражений COVID-19
891,headline | В Грузии за сутки выявлено 337 случ...,Заражений COVID-19
892,headline | Число заражений COVID-19 в мире при...,Заражений COVID-19
893,headline | В Казахстане выявлен 51 новый случа...,Заражений COVID-19
894,headline | В Казахстане увеличился суточный пр...,Заражений COVID-19


In [1]:
!nvidia-smi

Sat Aug 21 07:15:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
predictions

,Generated Text,Actual Text
0,Протесты в Молдавии,Политический кризис в Молдавии
1,Блокировка парламента Молдавии,Политический кризис в Молдавии
2,Протесты в Молдавии,Политический кризис в Молдавии
3,Протесты в Молдавии,Политический кризис в Молдавии
4,Додон: о кризисе в Молдавии,Политический кризис в Молдавии


In [1]:
!pip install -Uq spacy

In [55]:
!python -m spacy download ru_core_news_sm

     |████████████████████████████████| 16.1 MB 305 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [19]:
import spacy

nlp = spacy.load('ru_core_news_sm',  disable=["parser", "ner"])

def get_lemmas(text): return [token.lemma_ for token in nlp(text)]

In [123]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer()
tfidf.fit(train_df[train_df['is_valid'] == False]['X'])
predictions = pd.read_csv('outputs_headlines/predictions.csv', index_col=0)
gts = predictions["Actual Text"].unique()

feature_names = np.array(tfidf.get_feature_names())

def get_top_tf_idf_words(response, top_n=2):
    sorted_nzs = np.argsort(response.data)[:-(top_n+1):-1]
    return feature_names[response.indices[sorted_nzs]]

dl_results = []

for gt in gts:
  curr_preds = predictions[predictions['Actual Text'] == gt]["Generated Text"]
  t_text = tfidf.transform(['. '.join(curr_preds.values)])

  top_words = get_top_tf_idf_words(t_text, 2)
  variants = []
  for pred in curr_preds:
    pred_words = pred.lower().split(' ')
    i = len(set(top_words).intersection(set(pred_words)))
    if len(pred_words) > 1:
      i /= len(pred_words)
    if ' о ' in pred or ' об ' in pred:
      i *= .75
    variants.append((i, pred, gt))
  res = sorted(variants, reverse=True)[0]
  dl_results.append((res[1:]))

In [125]:
dl_df = pd.DataFrame(dl_results, columns=["result", "title"])
dl_df

,result,title
0,Додон об правительстве Молдавии,Политический кризис в Молдавии
1,SpaceX стартовала на орбиту,Запуск спутников SpaceX
2,Количество заражений COVID-19 в Молдавии,COVID-19 в Молдавии
3,Минэкономразвития,Экспортные контракты на зерно
4,Акции в Петербурге,Участники несогласованной акции в Петербурге
5,Въезд в РФ,Авиасообщение РФ с Белоруссией и Арменией
6,В Новосибирске землетрясения,Землетрясение в Монголии
7,Блокировка аккаунта Трампа,Трамп в соцсетях
8,Газоснабжение отключенных потребителей,Аварии на магистралях Газпрома
9,В Белоруссии арестовали Борисова,Акции оппозиции в Белоруссии


In [76]:
pd.concat([easy_result, dl_df])

,title,result
0,Значения фондовых индексов,Значения основных фондовых индексов
1,Рынок акций РФ,Рынок акций РФ утром
2,Кросс-курсы мировых валют,Кросс-курсы основных мировых валют
3,Вакцина от COVID-19,Бахрейн стал первой
4,Курсы валют,"Курс доллара расчетами ""завтра"""
5,Кросс-курсы валют,Кросс-курсы основных мировых валют
6,Итоги торгов акциями на Московской бирже,Результаты торгов акциями
7,Значения индексов Мосбиржи и РТС,Значения индексов Мосбиржи и РТС
8,Торги ADR и GDR российских компаний,Торги ADR и GDR российских
9,Кросс-курсы основных мировых валют,Кросс-курсы основных мировых валют


In [106]:
len(train_df)

1022